In [1]:
import duckdb as db
import pandas as pd
import numpy as np

def sql(query, database = "../data/database.duckdb"):
    con = db.connect(database)
    res = con.execute(query).df()
    con.close()
    return res

In [297]:
#Conversion de notre base de donnée acquise via SQL en DataFrame
data = pd.DataFrame(sql("""
    SELECT
        *
    FROM
       dataframe_view
    """))

ANALYSE EXPLORATOIRE DES DONNEES

In [298]:
#Creation d'un DF_Copy pour ne pas modifier le DF d'origine
data_copy = data

CLEAN DATA AVANT ML

In [299]:
def clean_row(row):
    return row.replace("[", "").replace("]", "").replace("'", "")
data_copy.drop(columns= ['Profit','Revenue', 'Budget'], inplace=True) #trop de valeurs manquantes, DF faussé
data_copy['ProductionCountries'] = data_copy['ProductionCountries'].apply(lambda x: 'Unknown' if x =='\\N' else x)
data_copy['Writers'] = data_copy['Writers'].apply(lambda x: 'Unknown' if x== '\\N'  else x)
data_copy['Resume'] = data_copy['Resume'].apply(lambda x: 'Unknown' if x == '\\N'  else x)
data_copy['Poster'] = data_copy['Poster'].apply(lambda x: 'Unknown' if x == '\\N'  else x)
#nettoyage de la colonne Year (il semblerait qu'il y ait qu'une ligne avec une valeur '\N')
data_copy['Year'] = data_copy['Year'].replace('\\N', np.nan)
data_copy = data_copy.dropna(subset=['Year'])
#On convertit la colonne Year en int pour pouvoir l'utiliser dans le modèle
data_copy['Year'] = data_copy['Year'].astype(int)
data_copy['Resume'].fillna('Unknown', inplace=True)
df_runtime = data_copy[data_copy['Runtime'] != '\\N']
df_runtime['Runtime'] = df_runtime['Runtime'].astype(int)
data_copy['Runtime'] = data_copy['Runtime'].apply(lambda x: df_runtime['Runtime'].median() if x == '\\N' else x)
data_copy['Runtime'] = data_copy['Runtime'].astype(int)
data_copy['Runtime'].describe()
data_copy['ProductionCountries'].fillna('Unknown', inplace=True)
data_copy['ProductionCountries'] = data_copy['ProductionCountries'].apply(clean_row)
data_copy['ProductionCompanies'] = data_copy['ProductionCompanies'].apply(clean_row)
data_copy['ProductionCountries'] = data_copy['ProductionCountries'].apply(
    lambda x: ', '.join(dict.fromkeys(str(x).split(', ')))
)
data_copy['Writers'].fillna('Unknown', inplace=True)

C:\Users\antoi\AppData\Local\Temp\ipykernel_28480\3720335642.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_copy['Year'] = data_copy['Year'].astype(int)
C:\Users\antoi\AppData\Local\Temp\ipykernel_28480\3720335642.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_copy[

In [253]:
data['Resume'].sample(5)

67280    The story of a teacher and his students, set i...
25737    In the aftermath of the most devastating confl...
39149    Whilst on their way to a concert four friends ...
26623    To Jane Falbury's New England farm comes a tro...
96840    A laid-back countryside detective Byung-su tak...
Name: Resume, dtype: object

In [140]:
data_copy.sample(2)

,tconst,Title,Genres,Year,Runtime,Actors,Director,Writers,Resume,Rating,Votes,Budget,Revenue,Profit,ProductionCompanies,ProductionCountries,Poster
98662,tt8956872,Valhalla: The Legend of Thor,"Adventure,Fantasy,Thriller",2019,105,"Ali Sivandi, Lára Jóhanna Jónsdóttir, Asbjørn ...",Fenar Ahmad,"Fenar Ahmad, Peter Madsen, Adam August",The Viking children Røskva and Tjalfe embark o...,5.2,3846,5256817,70821,-5185996,"['Profile Pictures', 'Nordisk Film', 'Bacon', ...","['DK', '', 'NO', '', 'IS', '', 'NO', 'DK', 'DK...",/lVGMUzFfmASySiIL52GVkzoREpW.jpg
82094,tt11854240,Appunti di un venditore di donne,"Drama,Thriller",2021,125,"Libero De Rienzo, Claudio Bigagli, Miriam Dalm...",Fabio Resinaro,"Fabio Resinaro, Giorgio Faletti","Set in 1978 Milan, Bravo lives his life among ...",6.3,444,0,0,0,"['Èliseo Entertainment', 'RAI']","['IT', 'IT']",/oj5wcSTwYBT838q9zCSIVuhXVTO.jpg


In [179]:
data_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101563 entries, 0 to 101562
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   tconst               101563 non-null  object 
 1   Title                101563 non-null  object 
 2   Genres               101563 non-null  object 
 3   Year                 101562 non-null  object 
 4   Runtime              101563 non-null  object 
 5   Actors               101563 non-null  object 
 6   Director             101563 non-null  object 
 7   Writers              101563 non-null  object 
 8   Resume               101563 non-null  object 
 9   Rating               101563 non-null  float64
 10  Votes                101563 non-null  int64  
 11  Budget               101563 non-null  int64  
 12  Revenue              101563 non-null  int64  
 13  Profit               101563 non-null  int64  
 14  ProductionCompanies  101563 non-null  object 
 15  ProductionCountri

In [255]:
#60111 valeurs manquantes dans la colonne ProductionCountries
data_copy['ProductionCountries'].isna().sum()
#On remplace les valeurs manquantes par une ['Unknown'] pour garder la structure de la colonne
data_copy['ProductionCountries'] = data_copy['ProductionCountries'].apply(lambda x: 'Unknown' if x =='\\N' else x)

In [257]:
data_copy['Resume'].sample(10)

76095    Gatti focuses on two men in a German concentra...
97630    75-year-old Giuseppe De Metrio has spent 30 ye...
16294    Siddhartha is the story of a young Brahmin and...
23571    A man's 10-year search for the underlying caus...
91869    Edgar Allan Poe's macabre masterpiece of murde...
23412    This is a sensitive film about human solidarit...
50012    The story of the Three Musketeers, D'Artagnan,...
48043    In 1970, a man is tortured and murdered by the...
79733    Tells the story of young Daniela ‘Blue Jeans’ ...
15911    In the 1970s a music promoter plucks Siamese t...
Name: Resume, dtype: object

PREMIER ESSAI d'UN KNN SUR YEAR POUR ESSAYER L'ALGO

In [10]:
#import de la bibliothèque scikit-learn
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer  



In [258]:
#nettoyage de la colonne Year (il semblerait qu'il y ait qu'une ligne avec une valeur '\N')
data_copy['Year'] = data_copy['Year'].replace('\\N', np.nan)
data_copy = data_copy.dropna(subset=['Year'])
#On convertit la colonne Year en int pour pouvoir l'utiliser dans le modèle
data_copy['Year'] = data_copy['Year'].astype(int)
#On vérifie que la colonne Year ne contient plus de valeurs manquantes
data_copy.info()


<class 'pandas.core.frame.DataFrame'>
Index: 101562 entries, 0 to 101562
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   tconst               101562 non-null  object 
 1   Title                101562 non-null  object 
 2   Genres               101562 non-null  object 
 3   Year                 101562 non-null  int64  
 4   Runtime              101562 non-null  object 
 5   Actors               101562 non-null  object 
 6   Director             101562 non-null  object 
 7   Writers              99411 non-null   object 
 8   Resume               100411 non-null  object 
 9   Rating               101562 non-null  float64
 10  Votes                101562 non-null  int64  
 11  Budget               101562 non-null  int64  
 12  Revenue              101562 non-null  int64  
 13  Profit               101562 non-null  int64  
 14  ProductionCompanies  101562 non-null  object 
 15  ProductionCountries  4

C:\Users\antoi\AppData\Local\Temp\ipykernel_28480\3438241628.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_copy['Year'] = data_copy['Year'].astype(int)


In [28]:
#Tentative de KNN sur Year
X_year = data_copy[['Year']]

knn = NearestNeighbors(n_neighbors=5, metric='euclidean')
knn.fit(X_year)

distances, indices = knn.kneighbors(X_year.iloc[[0]])
print("Distances:", distances)
print("Indices:", indices)

Distances: [[0. 0. 0. 0. 0.]]
Indices: [[ 57 202 203   0  84]]


les résultats indiquent que le knn fonctionnent mais ne sert à rien puisque basé que sur des années 

une fois les indices trouvés : boucle pour afficher le poster

In [49]:
pd.options.display.max_rows = 999
url = 'https://image.tmdb.org/t/p/w185'
for i in indices[0]:
    a = url+data_copy['Poster'].iloc[i]
    print(a)


https://image.tmdb.org/t/p/w185/cXOLaxcNjNAYmEx1trZxOTKhK3Q.jpg
https://image.tmdb.org/t/p/w185/uluhlXubGu1VxU63X9VHCLWDAYP.jpg
https://image.tmdb.org/t/p/w185/momkKuWburNTqKBF6ez7rvhYVhE.jpg
https://image.tmdb.org/t/p/w185/jRXYjXNq0Cs2TcJjLkki24MLp7u.jpg
https://image.tmdb.org/t/p/w185/z7uo9zmQdQwU5ZJHFpv2Upl30i1.jpg


In [48]:
print(url+data_copy['Poster'].iloc[97])
print(url+data_copy['Poster'].iloc[202])
print(indices[0])

https://image.tmdb.org/t/p/w185/j5jM5pq78ObAXX1WhTsb115EkLl.jpg
https://image.tmdb.org/t/p/w185/uluhlXubGu1VxU63X9VHCLWDAYP.jpg
[ 57 202 203   0  84]


tentative avec Realisator : pas possible, il faut encoder les données, KNN n'accepte que les données numériques

In [259]:
data_copy.select_dtypes('number').columns

Index(['Year', 'Rating', 'Votes', 'Budget', 'Revenue', 'Profit'], dtype='object')

sur les colonnes numériques (pas très représentatif car DF classé par recettes)

In [51]:
X_num = data_copy.select_dtypes('number').drop(columns=['Revenue'])

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', NearestNeighbors(n_neighbors=5, metric='euclidean'))
])


pipeline.fit(X_num)

distances, indices = pipeline.named_steps['knn'].kneighbors(X_num.iloc[[0]])
print("Distances:", distances)
print("Indices:", indices)
for i in indices:
    print(data_copy['Title'].iloc[i])

Distances: [[2.69713922e+09 2.69713923e+09 2.69713923e+09 2.69713924e+09
  2.69713924e+09]]
Indices: [[0 1 2 3 4]]
0                                        Avatar
1                             Avengers: Endgame
2                                       Titanic
3                      Avatar: The Way of Water
4    Star Wars: Episode VII - The Force Awakens
Name: Title, dtype: object


c:\Users\antoi\Documents\WCS\Projet-2\.venv\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(


In [60]:
X_rates_year = data_copy[['Rating', 'Votes', 'Year']]

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', NearestNeighbors(n_neighbors=10, metric='euclidean'))
])

film = 0
pipeline.fit(X_rates_year)

distances, indices = pipeline.named_steps['knn'].kneighbors(X_rates_year.iloc[[film]])
print("Distances:", distances)
print("Indices:", indices)
print(data_copy['Title'].iloc[film])
for i in indices[0]:
    print(data_copy['Title'].iloc[i])

Distances: [[1430988.99219377 1430989.33882759 1430994.50384651 1430997.84894571
  1430999.16358842 1430999.67765745 1430999.84301597 1431002.58017398
  1431003.05689887 1431003.31625462]]
Indices: [[6289   54   94 2490  111  145  581  227  484   63]]
Avatar
The Shawshank Redemption
The Dark Knight
Inception
Fight Club
Forrest Gump
Interstellar
Pulp Fiction
The Matrix
The Godfather
The Lord of the Rings: The Fellowship of the Ring


c:\Users\antoi\Documents\WCS\Projet-2\.venv\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(


UTILISATION DE TfidfVectorizer POUR LES COLONNES NON NUMERIQUES

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer

In [ ]:
X_text = data_copy[['Resume', 'Title']]

preprocessor = ColumnTransformer(
    transformers= [
        ('Resume', TfidfVectorizer(stop_words='english'), 'Resume'),
        ('Title', TfidfVectorizer(stop_words='english'), 'Title')
    ],
    )  

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('knn', NearestNeighbors(n_neighbors=5, metric='cosine'))

])

pipeline.fit(X_text)
distances, indices = pipeline.named_steps['knn'].kneighbors(X_text.iloc[[0]])
print("Distances:", distances)
print("Indices:", indices)

In [260]:
data_copy[['Resume','Title']].head()

,Resume,Title
0,"In the 22nd century, a paraplegic Marine is di...",Avatar
1,After the devastating events of Avengers: Infi...,Avengers: Endgame
2,101-year-old Rose DeWitt Bukater tells the sto...,Titanic
3,Set more than a decade after the events of the...,Avatar: The Way of Water
4,Thirty years after defeating the Galactic Empi...,Star Wars: Episode VII - The Force Awakens


In [264]:
data_copy['Resume'].apply(type).value_counts()
data_copy['Resume'].isna().sum()
data_copy['Resume'].fillna('Unknown', inplace=True)
data_copy['Resume'].isna().sum()

C:\Users\antoi\AppData\Local\Temp\ipykernel_28480\1950687486.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_copy['Resume'].fillna('Unknown', inplace=True)


np.int64(0)

In [ ]:
data_copy['Resume'] = data_copy['Resume'].apply(lambda x: x[0] if isinstance(x, list) else x)
X_text = data_copy[['Resume', 'Title']]

preprocessor = ColumnTransformer(
    transformers= [
        ('Resume', TfidfVectorizer(stop_words='english'), 'Resume'),
        ('Title', TfidfVectorizer(stop_words='english'), 'Title')
    ],
    )  

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('knn', NearestNeighbors(n_neighbors=5, metric='cosine'))

])

pipeline.fit(X_text)
vectorized_data = pipeline.named_steps['preprocessor'].transform(X_text)
distances, indices = pipeline.named_steps['knn'].kneighbors(vectorized_data[[0]])
print("Distances:", distances)
print("Indices:", indices)
for i in indices[0]:
    print(data_copy['Title'].iloc[i])
    print(data_copy['Resume'].iloc[i])
    print()

In [ ]:
data_copy['Resume'] = data_copy['Resume'].apply(lambda x: x[0] if isinstance(x, list) else x)
X_text = data_copy[['Resume', 'Title', 'Genres', 'Director', 'Rating', 'Votes']]

preprocessor = ColumnTransformer(
    transformers= [
        ('Resume', TfidfVectorizer(stop_words='english'), 'Resume'),
        ('Genres', TfidfVectorizer(stop_words='english'), 'Genres'),
        ('Title', TfidfVectorizer(stop_words='english'), 'Title'),
        ('Director', TfidfVectorizer(stop_words='english'), 'Director'),
        ('Rating', StandardScaler(), 'Rating'),
        ('Votes', StandardScaler(), 'Votes')
    ],
    )  

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('knn', NearestNeighbors(n_neighbors=5, metric='cosine'))

])
film_index = 5
pipeline.fit(X_text)
vectorized_data = pipeline.named_steps['preprocessor'].transform(X_text)
distances, indices = pipeline.named_steps['knn'].kneighbors(vectorized_data.getrow(film_index))
print("Distances:", distances)
print("Indices:", indices)
for i in indices[0]:
    print(data_copy['Title'].iloc[i])
    print(data_copy['Resume'].iloc[i])
    print(data_copy['Director'].iloc[i])
    print()

In [ ]:
num_cols = data_copy.select_dtypes('number')
non_num_cols = data_copy.select_dtypes(exclude='number')
non_num_cols.drop(columns=['tconst', 'Poster'], inplace=True)


In [296]:
num_cols[num_cols['Budget'] == 0]

,Year,Runtime,Rating,Votes,Budget,Revenue,Profit
83,2021,136,5.6,2886,0,686257563,686257563
91,2023,159,6.4,3697,0,673556758,673556758
150,2017,106,7.8,773484,0,527000000,527000000
186,2022,122,6.4,2022,0,460300583,460300583
193,2019,154,6.3,2393,0,450064993,450064993
...,...,...,...,...,...,...,...
91762,1999,84,4.1,611,0,0,0
91763,2017,90,7.5,510,0,0,0
91764,2022,71,6.8,270,0,0,0
91765,2021,95,7.7,1813,0,0,0


In [293]:
data_copy.iloc[101558:]

,tconst,Title,Genres,Year,Runtime,Actors,Director,Writers,Resume,Rating,Votes,Budget,Revenue,Profit,ProductionCompanies,ProductionCountries,Poster
101559,tt1074638,Skyfall,"Action,Adventure,Thriller",2012,143,"Rory Kinnear, Naomie Harris, Ralph Fiennes, Bé...",Sam Mendes,"Neal Purvis, John Logan, Robert Wade, Ian Fleming",When Bond's latest assignment goes gravely wro...,7.8,750850,200000000,0,-200000000,"Metro-Goldwyn-Mayer, Eon Productions, Danjaq, ...",Unknown,/d0IVecFQvsGdSbnMAHqiYsNYaJT.jpg
101560,tt9603212,Mission: Impossible - Dead Reckoning Part One,"Action,Adventure,Thriller",2023,163,"Vanessa Kirby, Ving Rhames, Pom Klementieff, S...",Christopher McQuarrie,"Christopher McQuarrie, Erik Jendresen, Bruce G...",The seventh installment of the Mission: Imposs...,7.6,287134,290000000,0,-290000000,"Paramount, Skydance Media, New Republic Pictur...",US,/pZCa8WgnP0AgVHsFjvuUDhcNjOJ.jpg
101561,tt1462764,Indiana Jones and the Dial of Destiny,"Action,Adventure,Sci-Fi",2023,154,"Thomas Kretschmann, Phoebe Waller-Bridge, Oliv...",James Mangold,"James Mangold, John-Henry Butterworth, Philip ...","Finding himself in a new era, approaching reti...",6.5,219921,294700000,0,-294700000,"Lucasfilm Ltd., Walt Disney Pictures, Paramount",Unknown,/sv7iVXYVJTtOUAM9KGwsIuO0dEc.jpg
101562,tt5433140,Fast X,"Action,Adventure,Crime",2023,141,"John Cena, Sung Kang, Jordana Brewster, Nathal...",Louis Leterrier,"Zach Dean, Justin Lin, Gary Scott Thompson, Da...",Over many missions and against impossible odds...,5.7,135401,340000000,0,-340000000,"Universal Pictures, Original Film, One Race, P...",US,/1E5baAaEse26fej7uHcjOgEE2t2.jpg
